In [1]:
# toggle to save space
mode = '_mean' if False else '_all'
print(mode)

_all


In [2]:
gbm_params = {
    'n_estimators' : 1500, # 500,  
    'max_depth' : -1,
    'learning_rate': 0.05,
    'bagging_fraction': 0.7, 
    
    'feature_fraction' : 0.9,
    'bagging_freq': 5,
    #'subsample' : 0.1,  # 
    #'subsample_freq' : 1,
    'num_leaves' : 31,
    'metric':'rmse',
    #'lambda_l1' : 1,  # Try defaults
    #'lambda_l2': 1, # Try defaults
    'verbose': 100
}

In [3]:
# TODO: add https://www.kaggle.com/corochann/optuna-tutorial-for-hyperparameter-optimization

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
import gc
from os import path
from sklearn.preprocessing import LabelEncoder
from pandas.core.dtypes.dtypes import CategoricalDtype
from tqdm import tqdm
from datetime import date 
import holidays
import lightgbm as lgb


warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

# label encoding
le = LabelEncoder()

In [5]:
class ConvertToDatetime(TransformerMixin):
        
    def transform(self, df, **transform_params):
        if 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [6]:
file_dtype = {
    'train': {'building_id': np.int16, 'meter': np.int8, 'meter_reading': np.float32},
    'test': {'building_id': np.int16, 'meter': np.int8},
    'building_metadata': {'site_id': np.int8, 'building_id': np.uint16, 'square_feet': np.float32, 'year_built': np.float16, 'floor_count': np.float16},
}

def loadFile(name):
    for dir_path in ['../input/ashrae-energy-prediction/','../input/_ashrae-energy-prediction/']:
        if path.exists(dir_path + name + '.csv'):
            return  ConvertToDatetime().transform(
                pd.read_csv(dir_path + name + '.csv', dtype=file_dtype[name]))
        


In [7]:
building = loadFile('building_metadata')
pre_train = loadFile('train')
#test = loadFile('test')

In [8]:
def merge(x):
    weather_processed_df = pd.read_pickle(f'../input/ashrae-energy-prediction-pickles/weather_processed{mode}.pickle')
    x = x.merge(building, on=['building_id'], how='left')
    gc.collect()
    x = x.merge(weather_processed_df, on=['site_id', 'timestamp'], how='left')
    gc.collect()
    x['site_id'] = x['site_id'].astype('int8');
    x['cloud_coverage'] = x['cloud_coverage'].astype('float16')
    gc.collect()
    return x
        
train = merge(pre_train) 
gc.collect()
print(train)
print('!!!! Warning we are missing weather for '+ str(train['air_temperature'].isnull().sum())+' rows')
train = train.dropna(axis=0, subset=['air_temperature'])
print(train.dtypes)

          building_id  meter           timestamp  meter_reading  site_id  \
0                   0      0 2016-01-01 00:00:00       0.000000        0   
1                   1      0 2016-01-01 00:00:00       0.000000        0   
2                   2      0 2016-01-01 00:00:00       0.000000        0   
3                   3      0 2016-01-01 00:00:00       0.000000        0   
4                   4      0 2016-01-01 00:00:00       0.000000        0   
5                   5      0 2016-01-01 00:00:00       0.000000        0   
6                   6      0 2016-01-01 00:00:00       0.000000        0   
7                   7      0 2016-01-01 00:00:00       0.000000        0   
8                   8      0 2016-01-01 00:00:00       0.000000        0   
9                   9      0 2016-01-01 00:00:00       0.000000        0   
10                 10      0 2016-01-01 00:00:00       0.000000        0   
11                 11      0 2016-01-01 00:00:00       0.000000        0   
12          

[20216100 rows x 113 columns]
!!!! Warning we are missing weather for 0 rows
building_id                               int16
meter                                      int8
timestamp                        datetime64[ns]
meter_reading                           float32
site_id                                    int8
primary_use                              object
square_feet                             float32
year_built                              float16
floor_count                             float16
air_temperature                         float16
dew_temperature                         float16
cloud_coverage                          float16
precip_depth_1_hr                       float16
wind_direction                          float16
wind_speed                              float16
sea_level_pressure                      float16
relative_humidity                       float16
air_temperature_mean_lag3               float16
air_temperature_max_lag3                float16
air_tempera

In [9]:
# See holiday notebook to generate, this is optional
holiday_df = None
if path.exists('../input/ashrae-energy-prediction-pickles/holiday_df.pickle'):
    holiday_df = pd.read_pickle('../input/ashrae-energy-prediction-pickles/holiday_df.pickle')
if holiday_df is not None:
    print(holiday_df.sample(20))

          site_id           timestamp  \
35994925        5 2018-04-02 03:00:00   
40967832        9 2017-05-29 08:00:00   
10070584       14 2016-07-04 01:00:00   
46181812       11 2017-09-04 07:00:00   
11637399        1 2016-08-01 00:00:00   
45887476       10 2018-07-04 22:00:00   
23017917        1 2017-12-25 14:00:00   
21499650        0 2018-02-19 14:00:00   
23408           3 2016-01-01 10:00:00   
54892693       14 2018-01-01 02:00:00   
46332284       11 2018-12-25 14:00:00   
22480653        1 2017-01-03 23:00:00   
54313237       14 2017-10-09 05:00:00   
11650335       12 2016-08-01 05:00:00   
33102413        3 2018-11-12 11:00:00   
18139062       13 2016-11-24 19:00:00   
22632798        1 2017-04-14 16:00:00   
39812960        8 2018-11-22 11:00:00   
17380263        6 2016-11-11 06:00:00   
59333921       15 2017-11-23 01:00:00   

                                                   holiday  
35994925  Easter Monday [England, Wales, Northern Ireland]  
40967832        

In [10]:
class AddTimeFeatures(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        # TODO: try week of year as numerical 
        df = df_a
        df['dayofweek'] = df['timestamp'].dt.dayofweek.astype('uint8') # vs weekend?
        #df['weekday'] = df['timestamp'].dt.weekday.astype('category')
        #df['dayofweek_hour'] = df['dayofweek_hour'].astype('category')
        #df['week'] = df['timestamp'].dt.week.astype('category')
        df['hour'] = df['timestamp'].dt.hour.astype('uint8')
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [11]:
class MeterReadingLog1p(TransformerMixin):
  
    def transform(self, df, **transform_params):
        if 'meter_reading' in df.columns:
            df['meter_reading_log1p'] = np.log1p(df['meter_reading'])
            df = df.drop('meter_reading', axis=1)
        return df
    
    def fit(self, X, y=None, **fit_params):
        return self
print(train.sample(20, random_state=42))
print(MeterReadingLog1p().transform(train.sample(20, random_state=42)))
gc.collect()

          building_id  meter           timestamp  meter_reading  site_id  \
14245562         1324      1 2016-09-16 16:00:00       0.000000       14   
1282718          1013      0 2016-01-24 06:00:00      32.000099       10   
13883790          229      1 2016-09-10 07:00:00     567.655029        2   
4781820           217      3 2016-04-01 01:00:00       0.000000        2   
10415393         1434      0 2016-07-10 04:00:00      65.750000       15   
1057008          1047      0 2016-01-20 04:00:00      90.983299       12   
4507399           911      1 2016-03-26 20:00:00     295.063995        9   
19478829         1039      0 2016-12-18 23:00:00      16.900000       12   
8955615           265      0 2016-06-14 06:00:00     128.369995        2   
13799839          896      0 2016-09-08 19:00:00     300.000000        9   
15647011          973      0 2016-10-11 11:00:00     247.000000        9   
2524294           813      0 2016-02-16 08:00:00      10.958300        8   
10016102    

[20 rows x 113 columns]
          building_id  meter           timestamp  site_id  \
14245562         1324      1 2016-09-16 16:00:00       14   
1282718          1013      0 2016-01-24 06:00:00       10   
13883790          229      1 2016-09-10 07:00:00        2   
4781820           217      3 2016-04-01 01:00:00        2   
10415393         1434      0 2016-07-10 04:00:00       15   
1057008          1047      0 2016-01-20 04:00:00       12   
4507399           911      1 2016-03-26 20:00:00        9   
19478829         1039      0 2016-12-18 23:00:00       12   
8955615           265      0 2016-06-14 06:00:00        2   
13799839          896      0 2016-09-08 19:00:00        9   
15647011          973      0 2016-10-11 11:00:00        9   
2524294           813      0 2016-02-16 08:00:00        8   
10016102          870      0 2016-07-03 02:00:00        8   
3915750           898      0 2016-03-15 03:00:00        9   
17217526          903      0 2016-11-08 09:00:00        9   


[20 rows x 113 columns]


24

In [12]:
class CreateMeterDescDF(TransformerMixin):

    def transform(self, df, **transform_params):
        global _building_meter_desc_DF
        if 'meter_reading_log1p' in df.columns:
            cols = ['mean']
            if mode == '_all':
                cols = ['mean','max','min','std']
            group = df.groupby(['building_id','meter'])['meter_reading_log1p']
            desc_DF = group.describe(percentiles=[.05, .25, .5, .75, .95 ])
            desc_DF = desc_DF.reset_index()
            col_dict = {}
            for col in desc_DF.columns:
                if col not in ['building_id', 'meter', 'count']:
                    col_dict[col] = 'meter_' + col
            _building_meter_desc_DF = desc_DF.rename(columns=col_dict).drop('count', axis=1)
            gc.collect()
        return df 
    def fit(self, X, y=None, **fit_params):
        return self

#if 'meter_mean' not in train.columns:
#    print(building_meter_desc_DF)
#    train = train.merge(building_meter_desc_DF, on=['building_id','meter'], how='left')
#    #test = test.merge(building_meter_desc_DF, on=['building_id','meter'], how='left')
#    del building_meter_desc_DF
CreateMeterDescDF().transform(
    AddTimeFeatures().transform(
        MeterReadingLog1p().transform(
            train.sample(2000, random_state=0)
        )
    )
)    
print(_building_meter_desc_DF)
gc.collect()

      building_id  meter  meter_mean  meter_std  meter_min  meter_5%  \
0               1      0    4.921724        NaN   4.921724  4.921724   
1               2      0    0.000000        NaN   0.000000  0.000000   
2               3      0    5.727568   0.128644   5.636602  5.645699   
3               5      0    0.000000        NaN   0.000000  0.000000   
4               6      0    0.000000        NaN   0.000000  0.000000   
5               8      0    5.930674        NaN   5.930674  5.930674   
6               9      1    5.300433   2.169901   3.766081  3.919516   
7              11      0    6.159797        NaN   6.159797  6.159797   
8              12      0    5.636602        NaN   5.636602  5.636602   
9              14      0    0.000000        NaN   0.000000  0.000000   
10             14      1    8.555042        NaN   8.555042  8.555042   
11             15      0    5.232920        NaN   5.232920  5.232920   
12             15      1    7.265242   0.810718   6.452293  6.53

[1349 rows x 11 columns]


21

In [13]:
class MergeMeterDescDF(TransformerMixin):
  
    def transform(self, df, **transform_params):
        # drop any columns to add
        dropCols =  [x for x in _building_meter_desc_DF.columns if x not in ['building_id', 'meter']]
        df = df.drop(dropCols, axis=1, errors='ignore') 
        return df.merge(_building_meter_desc_DF, on=['building_id','meter'], how='left')

    def fit(self, X, y=None, **fit_params):
        return self

print(MergeMeterDescDF().transform(train.sample(2000, random_state=0)))

      building_id  meter           timestamp  meter_reading  site_id  \
0             774      1 2016-08-07 08:00:00      36.128899        6   
1             206      0 2016-10-04 14:00:00     226.270004        2   
2            1269      0 2016-11-29 10:00:00      28.670799       14   
3             951      0 2016-10-10 04:00:00     113.000000        9   
4             656      0 2016-05-01 21:00:00      32.700001        5   
5              36      0 2016-06-05 19:00:00     178.830994        0   
6            1262      0 2016-07-19 05:00:00      73.739998       14   
7              52      0 2016-03-17 14:00:00       0.000000        0   
8            1133      2 2016-07-10 17:00:00     984.375000       13   
9            1123      0 2016-10-02 09:00:00      18.243999       13   
10           1237      0 2016-10-06 18:00:00      85.000000       14   
11            960      1 2016-11-17 10:00:00      90.066498        9   
12             11      0 2016-09-25 21:00:00     472.332001     

[2000 rows x 122 columns]


In [14]:
# "As you can see above, this data looks weired until May 20. It is 
# reported in this discussion by @barnwellguy that All electricity
# meter is 0 until May 20 for site_id == 0. Let's remove these data 
# from training data."
# https://www.kaggle.com/kaushal2896/ashrae-eda-fe-lightgbm-1-13
class RmS0M0(TransformerMixin):
  
    def transform(self, df, **transform_params):
        return df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

    def fit(self, X, y=None, **fit_params):
        return self
    


In [15]:
# TODO: write filter to remove any 0 meter reading that continue more than N days (try 3)
# Also we need to account for this by meter

In [16]:
# TODO: try rolling with power

In [17]:
    
    
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114483#latest-660771
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114874#latest-660970
class AddHolidays(TransformerMixin):
    def transform(self, df, **transform_params):
        if holiday_df is not None:
            df = df.merge(holiday_df, on=['timestamp','site_id'], how='left')
            df['holiday'] = df['holiday'].astype('category')
        else:
            print("Warning: Holiday DF is missing")
        return df

    def fit(self, X, y=None, **fit_params):
        return self
# Test 
if holiday_df is not None:
    print(holiday_df.columns)
    print(AddHolidays().transform(train.head(20))[['holiday','timestamp']])

Index(['site_id', 'timestamp', 'holiday'], dtype='object')
           holiday  timestamp
0   New Year's Day 2016-01-01
1   New Year's Day 2016-01-01
2   New Year's Day 2016-01-01
3   New Year's Day 2016-01-01
4   New Year's Day 2016-01-01
5   New Year's Day 2016-01-01
6   New Year's Day 2016-01-01
7   New Year's Day 2016-01-01
8   New Year's Day 2016-01-01
9   New Year's Day 2016-01-01
10  New Year's Day 2016-01-01
11  New Year's Day 2016-01-01
12  New Year's Day 2016-01-01
13  New Year's Day 2016-01-01
14  New Year's Day 2016-01-01
15  New Year's Day 2016-01-01
16  New Year's Day 2016-01-01
17  New Year's Day 2016-01-01
18  New Year's Day 2016-01-01
19  New Year's Day 2016-01-01


In [18]:
class RmHolidays(TransformerMixin):
    def transform(self, df, **transform_params):
        if holiday_df is not None:
            df = df.merge(holiday_df, on=['timestamp','site_id'], how='left')
            df = df.drop(df[df['holiday'].notnull()].index)
            df = df.drop(['holiday'], axis=1)
            gc.collect()
        else:
            print("Warning: Holiday DF is missing")
        return df

    def fit(self, X, y=None, **fit_params):
        return self

# Test you should see the new years removed
#print(train.head(100000).merge(building, on='building_id', how='left').columns)
print(RmHolidays().transform(train.head(100000)))

       building_id  meter           timestamp  meter_reading  site_id  \
55121            0      0 2016-01-02 00:00:00       0.000000        0   
55122            1      0 2016-01-02 00:00:00       0.000000        0   
55123            2      0 2016-01-02 00:00:00       0.000000        0   
55124            3      0 2016-01-02 00:00:00       0.000000        0   
55125            4      0 2016-01-02 00:00:00       0.000000        0   
55126            5      0 2016-01-02 00:00:00       0.000000        0   
55127            6      0 2016-01-02 00:00:00       0.000000        0   
55128            7      0 2016-01-02 00:00:00       0.000000        0   
55129            8      0 2016-01-02 00:00:00       0.000000        0   
55130            9      0 2016-01-02 00:00:00       0.000000        0   
55131           10      0 2016-01-02 00:00:00       0.000000        0   
55132           11      0 2016-01-02 00:00:00       0.000000        0   
55133           12      0 2016-01-02 00:00:00      

[41839 rows x 113 columns]


In [19]:
class SetCatTypes(TransformerMixin):
    
    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col]= df[col].astype('category')
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [20]:
class LogSquareFeet(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['log_square_feet'] = np.float16(np.log(df['square_feet']))
        return df

    def fit(self, X, y=None, **fit_params):
        return self
print(building.head(20)['square_feet'])

0       7432.0
1       2720.0
2       5376.0
3      23685.0
4     116607.0
5       8000.0
6      27926.0
7     121074.0
8      60809.0
9      27000.0
10    370773.0
11     49073.0
12     37100.0
13     99380.0
14     86250.0
15     83957.0
16     54644.0
17     15250.0
18    111891.0
19     18717.0
Name: square_feet, dtype: float32


In [21]:
# TODO: Play with scaling cloud coverage

In [22]:
class DropCols(TransformerMixin):

    def __init__(self, drop_cols):
        self._drop_cols = drop_cols
        
    def transform(self, df, **transform_params):
        df = df.drop(self._drop_cols, axis=1)
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [23]:
class ImputeYearBuilt(TransformerMixin):

    def transform(self, df, **transform_params):
        # revisit the choice of median vs anything else
        tmp_df = train.drop_duplicates(['site_id','building_id'])[['site_id','building_id','year_built']]
        year_built_median = tmp_df['year_built'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in tmp_df.groupby(['site_id'])['year_built'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = i_median
            else:
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = year_built_median
        df['building_age'] = np.uint8(df['year_built']-1900)
        del tmp_df, year_built_median
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self
print(ImputeYearBuilt().transform(train.sample(20))['building_age'])

16519757    70
6846725     70
5721825     76
13083055    70
14058756    70
18366480    68
2519147     91
18872974    62
13679418    62
4826946     70
7291690     70
3100981     70
7234748     81
8038605     70
11768342    70
9102063     70
7669276     70
19997728    70
3689927     70
8461512     70
Name: building_age, dtype: uint8


In [24]:
class ImputeFloorCount(TransformerMixin):

    def transform(self, df, **transform_params):
        # revisit the choice of median vs anything else
        tmp_df = train.drop_duplicates(['site_id','building_id'])[['site_id','building_id','floor_count']]
        floors_median = tmp_df['floor_count'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in tmp_df.groupby(['site_id'])['floor_count'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['floor_count'].isnull()) & (df['site_id'] == i), 'floor_count'] = i_median
            else:
                df.loc[(df['floor_count'].isnull()) & (df['site_id'] == i), 'floor_count'] = floors_median
        del tmp_df, floors_median
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

print(ImputeFloorCount().transform(train.sample(20))['floor_count'])

16136778    3.0
12911452    3.0
3108273     3.0
2121567     3.0
12312204    4.0
2825927     3.0
13263158    3.0
3269051     3.0
2653331     3.0
2693679     3.0
11452982    5.0
19340592    3.0
20041421    6.0
16056870    3.0
13403510    3.0
6852430     3.0
3156192     3.0
2356816     3.0
17264320    3.0
2226411     3.0
Name: floor_count, dtype: float16


In [25]:
class AddMeterDummies(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        for i in range(4):
            df['_meter_'+str(i)] = (df['building_id'].isin(
                train.loc[train['meter'] == i].building_id.unique()))
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [26]:
class AddRelativeHumidity(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        # code here
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [27]:
class DropCols(TransformerMixin):

    def __init__(self, drop_cols):
        self._drop_cols = drop_cols
        
    def transform(self, df, **transform_params):
        df = df.drop(self._drop_cols, axis=1)
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [28]:
class MarkNaNs(TransformerMixin):
        
    def transform(self, df, **transform_params):
        for col in  df.columns[df.isna().any()].tolist():
            df['_' + col + '_nan' ] = df[col].isnull()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [29]:
class GC(TransformerMixin):
        
    def transform(self, df, **transform_params):
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [30]:
class MergeDescHDOW(TransformerMixin):
    def transform(self, df, **transform_params):
        desc_h_dow_df = pd.read_csv('desc_h_dow.csv', dtype={'building_id': np.int16, 'meter': np.int8, 'meter_h_d_mean':np.float32,
                                                     'meter_h_d_std':np.float32, 'meter_h_d_min':np.float32,
                                                     'meter_h_d_25%':np.float32, 'meter_h_d_50%':np.float32,
                                                     'meter_h_d_75%':np.float32, 'meter_h_d_max':np.float32 })
        df = df.merge(desc_h_dow_df, on=['building_id', 'meter', 'dayofweek','hour'], how='left')
        del desc_h_dow_df
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self


In [31]:
# declare model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_log_error, mean_squared_error
from lightgbm import LGBMRegressor


def rmsle(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(y, y_pred.clip(0)))

def rmse(y, y_pred):
    # hack to prevent negative numbers
    return mean_squared_error(y, y_pred.clip(0))

def rmsee(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(np.expm1(y.clip(0)), np.expm1(y_pred.clip(0))))
    
rmsle_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmse_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmsee_scorer = make_scorer(
    lambda y_true, y_pred : rmsee(y_true, y_pred), 
    greater_is_better=False)

def lbm_rmsle(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False

# rob's custome function to do RMSLE while in the log1p space
def lbm_rmslee(y_true, y_pred):
    return 'RMSLEE', np.sqrt(np.mean(np.power(y_pred - y_true, 2))), False



In [32]:
%%time

x_pre_pipes = Pipeline(
    steps=[
        ('meterReadingLog1p',MeterReadingLog1p()),
        ('rmS0M0', RmS0M0()),
        ('addTimeFeatures', AddTimeFeatures()),
        ('logSquareFeet', LogSquareFeet()),
        #('rmHolidays', RmHolidays()),
        #('addHolidays', AddHolidays()),
        #('createMeterDescDF', CreateMeterDescDF()), # note declares a globe variable to pass
        #('mergeMeterDescDF', MergeMeterDescDF()), # populates both test and train from global
        ('mergeDescHDOW', MergeDescHDOW()),
        ('setCatTypes', SetCatTypes(['building_id', 'site_id', 'meter', 'primary_use'])),
        ('GC', GC())
    ]
)

train = x_pre_pipes.transform(train)
print(train.columns)

Index(['building_id', 'meter', 'timestamp', 'site_id', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'dew_temperature',
       ...
       'hour', 'log_square_feet', 'Unnamed: 0', 'meter_h_d_mean',
       'meter_h_d_std', 'meter_h_d_min', 'meter_h_d_25%', 'meter_h_d_50%',
       'meter_h_d_75%', 'meter_h_d_max'],
      dtype='object', length=124)
Wall time: 26.5 s


In [33]:
# pre_a_pipes is for preprocessing that doesn't change impute
# values
x_fold_pipes = Pipeline(
    steps=[
        #('markNans',MarkNaNs()),
        #('convertToDatetime', ConvertToDatetime()),
        ('imputeYearBuilt', ImputeYearBuilt()),
        ('imputeFloorCount', ImputeFloorCount()),
        ('dropCols', DropCols(['timestamp','square_feet', 'year_built'])),
        ('GC', GC())
    ]
)

sample_train_X = x_fold_pipes.transform(train.sample(20))
print(sample_train_X.columns)
print(sample_train_X.dtypes)

Index(['building_id', 'meter', 'site_id', 'primary_use', 'floor_count',
       'air_temperature', 'dew_temperature', 'cloud_coverage',
       'precip_depth_1_hr', 'wind_direction',
       ...
       'log_square_feet', 'Unnamed: 0', 'meter_h_d_mean', 'meter_h_d_std',
       'meter_h_d_min', 'meter_h_d_25%', 'meter_h_d_50%', 'meter_h_d_75%',
       'meter_h_d_max', 'building_age'],
      dtype='object', length=122)
building_id                      category
meter                            category
site_id                          category
primary_use                      category
floor_count                       float16
air_temperature                   float16
dew_temperature                   float16
cloud_coverage                    float16
precip_depth_1_hr                 float16
wind_direction                    float16
wind_speed                        float16
sea_level_pressure                float16
relative_humidity                 float16
air_temperature_mean_lag3         flo

In [34]:
# this stratified strategy from
# https://www.kaggle.com/isaienkov/lightgbm-fe-1-19/notebook
# trying no shuffle https://www.kaggle.com/c/ashrae-energy-prediction/discussion/115851#latest-666115
folds = 5
kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=42)


In [35]:
def cvTrainMeterEnsemble(train, gbm_params):
    meter_models = [ [], [], [], [] ]
    for train_index, val_index in kf.split(train, train['building_id']):
        f_train = x_fold_pipes.transform(train.loc[train_index])
        f_val = x_fold_pipes.transform(train.loc[val_index])
        for i in range(4):
            print(f'training meter {i}')
            f_train_m = f_train[f_train['meter'] == i]
            f_val_m = f_val[f_val['meter'] == i]
            gbm_params_m = gbm_params
            if i == 0:
                gbm_params_m['learning_rate']=0.04
            elif i == 1:
                gbm_params_m['learning_rate']=0.06
                gbm_params_m['bagging_fraction']=0.5
            elif i == 2:
                gbm_params_m['learning_rate']=0.05
                gbm_params_m['bagging_fraction']=0.8
            else:
                gbm_params_m['learning_rate']=0.04
                gbm_params_m['bagging_fraction']=0.9                
            gbm = LGBMRegressor(**gbm_params_m)
            gbm.fit(f_train_m.drop('meter_reading_log1p', axis=1), f_train_m['meter_reading_log1p'],
                eval_set=[(f_val_m.drop('meter_reading_log1p', axis=1), f_val_m['meter_reading_log1p'])],
                # https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114722#latest-660848
                # eval_metric=lbm_rmslee,
                early_stopping_rounds=20)
            meter_models[i].append(gbm)
            del f_train_m, f_val_m, gbm
            gc.collect()
        del f_train, f_val
        gc.collect()
    return meter_models

In [36]:
%%time
meter_models = cvTrainMeterEnsemble(train, gbm_params)

training meter 0
[1]	valid_0's rmse: 1.52581
Training until validation scores don't improve for 20 rounds
[2]	valid_0's rmse: 1.47493
[3]	valid_0's rmse: 1.42651
[4]	valid_0's rmse: 1.38036
[5]	valid_0's rmse: 1.33586
[6]	valid_0's rmse: 1.29397
[7]	valid_0's rmse: 1.25405
[8]	valid_0's rmse: 1.21626
[9]	valid_0's rmse: 1.18047
[10]	valid_0's rmse: 1.14636
[11]	valid_0's rmse: 1.11408
[12]	valid_0's rmse: 1.08311
[13]	valid_0's rmse: 1.05402
[14]	valid_0's rmse: 1.0265
[15]	valid_0's rmse: 1.00042
[16]	valid_0's rmse: 0.97602
[17]	valid_0's rmse: 0.952526
[18]	valid_0's rmse: 0.930672
[19]	valid_0's rmse: 0.910183
[20]	valid_0's rmse: 0.890224
[21]	valid_0's rmse: 0.872017
[22]	valid_0's rmse: 0.855237
[23]	valid_0's rmse: 0.839168
[24]	valid_0's rmse: 0.824319
[25]	valid_0's rmse: 0.809298
[26]	valid_0's rmse: 0.795154
[27]	valid_0's rmse: 0.783009
[28]	valid_0's rmse: 0.771664
[29]	valid_0's rmse: 0.761101
[30]	valid_0's rmse: 0.750203
[31]	valid_0's rmse: 0.740769
[32]	valid_0's rms

[27]	valid_0's rmse: 1.59752
[28]	valid_0's rmse: 1.58676
[29]	valid_0's rmse: 1.57723
[30]	valid_0's rmse: 1.56679
[31]	valid_0's rmse: 1.55848
[32]	valid_0's rmse: 1.55033
[33]	valid_0's rmse: 1.5444
[34]	valid_0's rmse: 1.53891
[35]	valid_0's rmse: 1.53138
[36]	valid_0's rmse: 1.52546
[37]	valid_0's rmse: 1.51928
[38]	valid_0's rmse: 1.51559
[39]	valid_0's rmse: 1.50942
[40]	valid_0's rmse: 1.50586
[41]	valid_0's rmse: 1.50309
[42]	valid_0's rmse: 1.50109
[43]	valid_0's rmse: 1.49595
[44]	valid_0's rmse: 1.4938
[45]	valid_0's rmse: 1.49134
[46]	valid_0's rmse: 1.4887
[47]	valid_0's rmse: 1.48568
[48]	valid_0's rmse: 1.48391
[49]	valid_0's rmse: 1.48188
[50]	valid_0's rmse: 1.48059
[51]	valid_0's rmse: 1.47848
[52]	valid_0's rmse: 1.47691
[53]	valid_0's rmse: 1.47531
[54]	valid_0's rmse: 1.47464
[55]	valid_0's rmse: 1.47371
[56]	valid_0's rmse: 1.47399
[57]	valid_0's rmse: 1.47228
[58]	valid_0's rmse: 1.4723
[59]	valid_0's rmse: 1.47051
[60]	valid_0's rmse: 1.47025
[61]	valid_0's rms

[152]	valid_0's rmse: 1.49712
[153]	valid_0's rmse: 1.4969
[154]	valid_0's rmse: 1.49692
[155]	valid_0's rmse: 1.49702
[156]	valid_0's rmse: 1.49726
[157]	valid_0's rmse: 1.49759
[158]	valid_0's rmse: 1.49756
[159]	valid_0's rmse: 1.4971
[160]	valid_0's rmse: 1.49734
[161]	valid_0's rmse: 1.49694
[162]	valid_0's rmse: 1.49666
[163]	valid_0's rmse: 1.49651
[164]	valid_0's rmse: 1.49659
[165]	valid_0's rmse: 1.49648
[166]	valid_0's rmse: 1.49632
[167]	valid_0's rmse: 1.4963
[168]	valid_0's rmse: 1.49618
[169]	valid_0's rmse: 1.49602
[170]	valid_0's rmse: 1.49557
[171]	valid_0's rmse: 1.49587
[172]	valid_0's rmse: 1.4956
[173]	valid_0's rmse: 1.49549
[174]	valid_0's rmse: 1.49519
[175]	valid_0's rmse: 1.49559
[176]	valid_0's rmse: 1.49558
[177]	valid_0's rmse: 1.49574
[178]	valid_0's rmse: 1.49564
[179]	valid_0's rmse: 1.49586
[180]	valid_0's rmse: 1.49621
[181]	valid_0's rmse: 1.49587
[182]	valid_0's rmse: 1.49592
[183]	valid_0's rmse: 1.49612
[184]	valid_0's rmse: 1.49638
[185]	valid_0'

[105]	valid_0's rmse: 1.14998
[106]	valid_0's rmse: 1.15021
[107]	valid_0's rmse: 1.15023
[108]	valid_0's rmse: 1.15027
[109]	valid_0's rmse: 1.15007
[110]	valid_0's rmse: 1.15006
[111]	valid_0's rmse: 1.14999
[112]	valid_0's rmse: 1.15002
[113]	valid_0's rmse: 1.15017
[114]	valid_0's rmse: 1.14963
[115]	valid_0's rmse: 1.14965
[116]	valid_0's rmse: 1.14971
[117]	valid_0's rmse: 1.14915
[118]	valid_0's rmse: 1.14899
[119]	valid_0's rmse: 1.14877
[120]	valid_0's rmse: 1.14863
[121]	valid_0's rmse: 1.14842
[122]	valid_0's rmse: 1.14822
[123]	valid_0's rmse: 1.14865
[124]	valid_0's rmse: 1.1485
[125]	valid_0's rmse: 1.14842
[126]	valid_0's rmse: 1.14797
[127]	valid_0's rmse: 1.148
[128]	valid_0's rmse: 1.1482
[129]	valid_0's rmse: 1.14818
[130]	valid_0's rmse: 1.1481
[131]	valid_0's rmse: 1.14812
[132]	valid_0's rmse: 1.14801
[133]	valid_0's rmse: 1.14793
[134]	valid_0's rmse: 1.14791
[135]	valid_0's rmse: 1.14795
[136]	valid_0's rmse: 1.14841
[137]	valid_0's rmse: 1.1484
[138]	valid_0's 

[42]	valid_0's rmse: 1.49692
[43]	valid_0's rmse: 1.49372
[44]	valid_0's rmse: 1.49172
[45]	valid_0's rmse: 1.48954
[46]	valid_0's rmse: 1.4876
[47]	valid_0's rmse: 1.48449
[48]	valid_0's rmse: 1.48329
[49]	valid_0's rmse: 1.48266
[50]	valid_0's rmse: 1.48082
[51]	valid_0's rmse: 1.48012
[52]	valid_0's rmse: 1.47956
[53]	valid_0's rmse: 1.47882
[54]	valid_0's rmse: 1.47746
[55]	valid_0's rmse: 1.47633
[56]	valid_0's rmse: 1.47587
[57]	valid_0's rmse: 1.47494
[58]	valid_0's rmse: 1.47276
[59]	valid_0's rmse: 1.47213
[60]	valid_0's rmse: 1.47048
[61]	valid_0's rmse: 1.47015
[62]	valid_0's rmse: 1.46973
[63]	valid_0's rmse: 1.469
[64]	valid_0's rmse: 1.4684
[65]	valid_0's rmse: 1.4674
[66]	valid_0's rmse: 1.467
[67]	valid_0's rmse: 1.46587
[68]	valid_0's rmse: 1.46571
[69]	valid_0's rmse: 1.46546
[70]	valid_0's rmse: 1.46437
[71]	valid_0's rmse: 1.46436
[72]	valid_0's rmse: 1.46351
[73]	valid_0's rmse: 1.4629
[74]	valid_0's rmse: 1.46318
[75]	valid_0's rmse: 1.46338
[76]	valid_0's rmse: 1

...

[69]	valid_0's rmse: 1.37679
[70]	valid_0's rmse: 1.37449
[71]	valid_0's rmse: 1.37415
[72]	valid_0's rmse: 1.37282
[73]	valid_0's rmse: 1.37243
[74]	valid_0's rmse: 1.37094
[75]	valid_0's rmse: 1.3687
[76]	valid_0's rmse: 1.36877
[77]	valid_0's rmse: 1.36848
[78]	valid_0's rmse: 1.36777
[79]	valid_0's rmse: 1.36704
[80]	valid_0's rmse: 1.36496
[81]	valid_0's rmse: 1.36367
[82]	valid_0's rmse: 1.36277
[83]	valid_0's rmse: 1.36244
[84]	valid_0's rmse: 1.36211
[85]	valid_0's rmse: 1.36158
[86]	valid_0's rmse: 1.35938
[87]	valid_0's rmse: 1.35954
[88]	valid_0's rmse: 1.35862
[89]	valid_0's rmse: 1.35728
[90]	valid_0's rmse: 1.35572
[91]	valid_0's rmse: 1.35461
[92]	valid_0's rmse: 1.35484
[93]	valid_0's rmse: 1.35464
[94]	valid_0's rmse: 1.35367
[95]	valid_0's rmse: 1.35299
[96]	valid_0's rmse: 1.35334
[97]	valid_0's rmse: 1.35309
[98]	valid_0's rmse: 1.35216
[99]	valid_0's rmse: 1.3494
[100]	valid_0's rmse: 1.3489
[101]	valid_0's rmse: 1.34851
[102]	valid_0's rmse: 1.34833
[103]	valid_0'

[194]	valid_0's rmse: 1.20069
[195]	valid_0's rmse: 1.20084
[196]	valid_0's rmse: 1.20023
[197]	valid_0's rmse: 1.20028
Early stopping, best iteration is:
[177]	valid_0's rmse: 1.19959
training meter 0
[1]	valid_0's rmse: 1.56773
Training until validation scores don't improve for 20 rounds
[2]	valid_0's rmse: 1.51741
[3]	valid_0's rmse: 1.4694
[4]	valid_0's rmse: 1.42413
[5]	valid_0's rmse: 1.38072
[6]	valid_0's rmse: 1.33944
[7]	valid_0's rmse: 1.30031
[8]	valid_0's rmse: 1.26305
[9]	valid_0's rmse: 1.22788
[10]	valid_0's rmse: 1.19461
[11]	valid_0's rmse: 1.16294
[12]	valid_0's rmse: 1.13289
[13]	valid_0's rmse: 1.10431
[14]	valid_0's rmse: 1.07764
[15]	valid_0's rmse: 1.05204
[16]	valid_0's rmse: 1.0281
[17]	valid_0's rmse: 1.00529
[18]	valid_0's rmse: 0.984252
[19]	valid_0's rmse: 0.964228
[20]	valid_0's rmse: 0.945114
[21]	valid_0's rmse: 0.927385
[22]	valid_0's rmse: 0.910823
[23]	valid_0's rmse: 0.895257
[24]	valid_0's rmse: 0.881063
[25]	valid_0's rmse: 0.867177
[26]	valid_0's 

[56]	valid_0's rmse: 1.41023
[57]	valid_0's rmse: 1.40998
[58]	valid_0's rmse: 1.40997
[59]	valid_0's rmse: 1.40986
[60]	valid_0's rmse: 1.40981
[61]	valid_0's rmse: 1.40754
[62]	valid_0's rmse: 1.40737
[63]	valid_0's rmse: 1.40689
[64]	valid_0's rmse: 1.40702
[65]	valid_0's rmse: 1.40723
[66]	valid_0's rmse: 1.40706
[67]	valid_0's rmse: 1.40702
[68]	valid_0's rmse: 1.40687
[69]	valid_0's rmse: 1.40706
[70]	valid_0's rmse: 1.40672
[71]	valid_0's rmse: 1.4066
[72]	valid_0's rmse: 1.40698
[73]	valid_0's rmse: 1.407
[74]	valid_0's rmse: 1.40683
[75]	valid_0's rmse: 1.40563
[76]	valid_0's rmse: 1.40502
[77]	valid_0's rmse: 1.40508
[78]	valid_0's rmse: 1.40548
[79]	valid_0's rmse: 1.40546
[80]	valid_0's rmse: 1.40553
[81]	valid_0's rmse: 1.4057
[82]	valid_0's rmse: 1.40585
[83]	valid_0's rmse: 1.40625
[84]	valid_0's rmse: 1.40528
[85]	valid_0's rmse: 1.40508
[86]	valid_0's rmse: 1.40486
[87]	valid_0's rmse: 1.4041
[88]	valid_0's rmse: 1.40517
[89]	valid_0's rmse: 1.40525
[90]	valid_0's rmse

[201]	valid_0's rmse: 1.50884
training meter 3
[1]	valid_0's rmse: 2.6692
Training until validation scores don't improve for 20 rounds
[2]	valid_0's rmse: 2.60081
[3]	valid_0's rmse: 2.536
[4]	valid_0's rmse: 2.47366
[5]	valid_0's rmse: 2.41414
[6]	valid_0's rmse: 2.35842
[7]	valid_0's rmse: 2.30579
[8]	valid_0's rmse: 2.25463
[9]	valid_0's rmse: 2.20857
[10]	valid_0's rmse: 2.16403
[11]	valid_0's rmse: 2.12252
[12]	valid_0's rmse: 2.08316
[13]	valid_0's rmse: 2.04534
[14]	valid_0's rmse: 2.01081
[15]	valid_0's rmse: 1.97869
[16]	valid_0's rmse: 1.94894
[17]	valid_0's rmse: 1.92183
[18]	valid_0's rmse: 1.8947
[19]	valid_0's rmse: 1.86943
[20]	valid_0's rmse: 1.84449
[21]	valid_0's rmse: 1.82175
[22]	valid_0's rmse: 1.80201
[23]	valid_0's rmse: 1.78333
[24]	valid_0's rmse: 1.76537
[25]	valid_0's rmse: 1.74822
[26]	valid_0's rmse: 1.7328
[27]	valid_0's rmse: 1.71777
[28]	valid_0's rmse: 1.70397
[29]	valid_0's rmse: 1.69128
[30]	valid_0's rmse: 1.6791
[31]	valid_0's rmse: 1.66799
[32]	val

[36]	valid_0's rmse: 1.27144
[37]	valid_0's rmse: 1.26873
[38]	valid_0's rmse: 1.26613
[39]	valid_0's rmse: 1.26315
[40]	valid_0's rmse: 1.26129
[41]	valid_0's rmse: 1.2604
[42]	valid_0's rmse: 1.25927
[43]	valid_0's rmse: 1.25935
[44]	valid_0's rmse: 1.2586
[45]	valid_0's rmse: 1.25726
[46]	valid_0's rmse: 1.25724
[47]	valid_0's rmse: 1.25689
[48]	valid_0's rmse: 1.25676
[49]	valid_0's rmse: 1.25535
[50]	valid_0's rmse: 1.25551
[51]	valid_0's rmse: 1.25611
[52]	valid_0's rmse: 1.25639
[53]	valid_0's rmse: 1.25627
[54]	valid_0's rmse: 1.25681
[55]	valid_0's rmse: 1.25753
[56]	valid_0's rmse: 1.25793
[57]	valid_0's rmse: 1.25833
[58]	valid_0's rmse: 1.25857
[59]	valid_0's rmse: 1.25933
[60]	valid_0's rmse: 1.25989
[61]	valid_0's rmse: 1.25854
[62]	valid_0's rmse: 1.25872
[63]	valid_0's rmse: 1.25923
[64]	valid_0's rmse: 1.26016
[65]	valid_0's rmse: 1.26011
[66]	valid_0's rmse: 1.26045
[67]	valid_0's rmse: 1.2609
[68]	valid_0's rmse: 1.26116
[69]	valid_0's rmse: 1.26124
Early stopping, b

In [37]:
for i in range(4):
    print('meter: '+ str(i))
    for model in meter_models[i]:
        print(model.best_score_['valid_0']['rmse'])


meter: 0
0.6379532658563773
0.5056721630678428
0.6086586367420809
0.6648635042397903
0.6306084524950294
meter: 1
1.2768475830978099
1.140374224680506
1.1722321559021645
1.4041041777519037
1.2553527268100055
meter: 2
1.4576609168450934
1.3847607788479235
1.3452873964413627
1.5088392979556622
1.3712170800371983
meter: 3
1.4938891359477513
1.5615765728697764
1.199592939772431
1.5179056683004453
1.6289816741927827


In [38]:
def createFeature_DF(model):
    imprtc_df = pd.DataFrame()
    imprtc_df['feature'] = sample_train_X.drop('meter_reading_log1p', axis=1).columns   
    imprtc_df['importance'] = model.feature_importances_
    imprtc_df.sort_values('importance', ascending=False, inplace= True)
    imprtc_df.sort_values('importance', ascending=False, inplace= True)
    return imprtc_df


# Importance rank for first model in cross val models
for i in range(4):
    print('meter: '+ str(i))
    print(createFeature_DF(meter_models[i][0]))


meter: 0
                           feature  importance
0                      building_id         553
113                 meter_h_d_mean         344
117                  meter_h_d_50%         220
118                  meter_h_d_75%         199
107    relative_humidity_min_lag96          64
28      precip_depth_1_hr_std_lag3          57
79       air_temperature_min_lag96          51
2                          site_id          48
112                     Unnamed: 0          48
86       dew_temperature_max_lag96          47
114                  meter_h_d_std          34
87       dew_temperature_min_lag96          32
78       air_temperature_max_lag96          30
116                  meter_h_d_25%          29
94    sea_level_pressure_max_lag96          25
84        cloud_coverage_std_lag96          23
85      dew_temperature_mean_lag96          21
105   relative_humidity_mean_lag96          21
98        wind_direction_max_lag96          21
77      air_temperature_mean_lag96          20
4   

[121 rows x 2 columns]
meter: 3
                          feature  importance
0                     building_id        2061
113                meter_h_d_mean         287
79      air_temperature_min_lag96         163
45     air_temperature_mean_lag72         136
77     air_temperature_mean_lag96         129
14       air_temperature_max_lag3         118
46      air_temperature_max_lag72         107
94   sea_level_pressure_max_lag96         100
78      air_temperature_max_lag96          90
107   relative_humidity_min_lag96          85
114                 meter_h_d_std          84
87      dew_temperature_min_lag96          80
86      dew_temperature_max_lag96          78
85     dew_temperature_mean_lag96          77
95   sea_level_pressure_min_lag96          75
13      air_temperature_mean_lag3          72
97      wind_direction_mean_lag96          69
15       air_temperature_min_lag3          66
100      wind_direction_std_lag96          64
75    relative_humidity_min_lag72          58
28

In [39]:
features_df = None
for i in range(4):
    print('meter: '+ str(i))
    for model in meter_models[i]:
        if features_df is None:
            features_df = createFeature_DF(model)
        else:
            features_df = features_df.append(createFeature_DF(model))

features_df = features_df.groupby('feature').sum().sort_values('importance', ascending=False).reset_index()
features_df['feature'] = features_df['feature'].str.pad(features_df['feature'].str.len().max(), side ='right') 

features_df.to_csv("featuress_multi_meter_light_GBM.csv")
print(features_df)
print()            

meter: 0
meter: 1
meter: 2
meter: 3
                           feature  importance
0    building_id                         32827
1    meter_h_d_mean                       5477
2    precip_depth_1_hr_std_lag3           2893
3    air_temperature_min_lag96            2548
4    meter_h_d_50%                        2120
5    meter_h_d_75%                        2112
6    meter_h_d_std                        1995
7    air_temperature_mean_lag72           1933
8    air_temperature_mean_lag96           1760
9    relative_humidity_min_lag96          1688
10   air_temperature_max_lag96            1682
11   sea_level_pressure_max_lag96         1615
12   dew_temperature_min_lag96            1605
13   dew_temperature_max_lag96            1503
14   air_temperature_min_lag3             1183
15   air_temperature_max_lag72            1098
16   sea_level_pressure_min_lag96         1083
17   log_square_feet                      1081
18   Unnamed: 0                           1057
19   meter_h_d_25%      

In [40]:
# %%time
# ## Single fit single model

# gbm = LGBMRegressor(**gbm_params)
# f_train_X, f_train_y = getInFoldXY(train.index)
# gbm.fit(f_train_X, f_train_y)

In [41]:
train_analysis = x_fold_pipes.transform(train)
train_analysis.to_pickle('train_analysis')
pickle.dump(meter_models, open( "meter_models_analysis.pickle", "wb" ))

In [42]:
test = loadFile('test')
test = merge(test)
test = x_pre_pipes.transform(test)
test = x_fold_pipes.transform(test)
test = test.drop('row_id', axis=1)


#print(test.sample(n=20,  random_state=42))
print(test.shape)
#print(test.dtypes)

(41697600, 121)


In [43]:
if train_analysis is None:
    train_analysis = pd.read_pickle('train_analysis')
    

In [44]:
l1 =  x_fold_pipes.transform(train).columns.tolist()
l2 =  test.columns.tolist()
print(list(x for x in l1 if x not in l2))
print(list(x for x in l2 if x not in l1))

['meter_reading_log1p']
[]


In [45]:
def predMeters(test_X):
    test_y = test_X[['meter']] 
    test_y['meter_reading_log1p'] = np.nan
    for i in range(4):
        X = test_X[test_X['meter'] == i]
        if X.shape[0] > 0:
            preds = np.expm1(sum([model.predict(X) for model in meter_models[i]])/folds)
            test_y.loc[test_y['meter'] == i, 'meter_reading_log1p'] = preds
    return test_y['meter_reading_log1p'].tolist()
    
print(predMeters(test.sample(n=20,  random_state=42)))    


[123.92972243706653, 10.173173503475002, 58.29844604776384, 1.866193464358147, 1205.810299287721, 8.439220715089784, 314.8765185159628, 1255.1560515237418, 253.1715049498708, 87.86877992009495, 47.54893130776593, 784.8028890794732, 854.5132327985867, 36.56740134591034, 62.7281056794045, 71.56181168650494, 17.43763539952613, 70.46121544964292, 60.437287960557164, 163.7998891277201]


In [46]:
# Predict using cross val models ensemble 
i=0
res=[]
step_size = 50000
for j in tqdm(range(int(np.ceil(test.shape[0]/50000)))):
    res.append(predMeters(test.iloc[i:i+step_size]))
    i+=step_size
    gc.collect()


100%|████████████████████████████████████████████████████████████████████████████████| 834/834 [34:44<00:00,  3.36s/it]


In [47]:
# Save using cross val models ensemble 
res = np.concatenate(res)
print(len(res))
submission = pd.read_csv('../input/ashrae-energy-prediction/sample_submission.csv')
submission['meter_reading'] = res
submission.loc[submission['meter_reading']<0, 'meter_reading'] = 0
submission.to_csv('submission_meter.csv.zip', index=False)
submission.shape

41697600


(41697600, 2)

In [48]:
# # Predict single model fit
# i=0
# res=[]
# step_size = 50000
# for j in tqdm(range(int(np.ceil(test_X.shape[0]/50000)))):
#    #res.append(np.expm1(sum([model.predict(test_X.iloc[i:i+step_size]) for model in models])/folds))
#    res.append(np.expm1(gbm.predict(test_X.iloc[i:i+step_size])))
#    i+=step_size
    